> **Rappel** : clique sur une cellule grise, puis **Shift + Entree** pour l'executer.
> Execute les cellules **dans l'ordre** de haut en bas.

---

# Leçon 2 : Apprendre de ses erreurs

## Le secret de l'IA : se tromper, corriger, recommencer

Imagine que tu apprends à lancer une balle dans un panier :
1. Tu lances -> tu rates à droite
2. Tu corriges un peu à gauche
3. Tu relances -> plus près !
4. Tu continues jusqu'à marquer

L'IA fait **exactement** pareil. Elle fait une prédiction, regarde si c'est
bon, et ajuste. Ça s'appelle **l'entraînement**.

---
## Étape 1 : Mesurer l'erreur

D'abord, il faut un moyen de dire **à quel point** le modèle s'est trompé.
On appelle ça la **loss** (perte en anglais).

- Loss haute = le modèle se trompe beaucoup
- Loss basse = le modèle devine bien

In [ ]:
import math

# Imaginons que le modèle prédit les probabilités suivantes
# pour la lettre qui suit 'p' dans 'pikachu' :

prediction = {
    "i": 0.6,  # 60% -> bonne réponse !
    "a": 0.2,  # 20%
    "e": 0.15,  # 15%
    "o": 0.05,  # 5%
}

# La bonne réponse est 'i'
bonne_reponse = "i"

# La loss = à quel point on est surpris par la bonne réponse
# Si on avait dit 100% pour 'i', la surprise serait de 0 (parfait !)
# Si on avait dit 1% pour 'i', la surprise serait énorme

loss = -math.log(prediction[bonne_reponse])
print(
    f"Le modèle donnait {prediction[bonne_reponse]:.0%} de chance à '{bonne_reponse}'"
)
print(f"Loss = {loss:.2f}")
print()

# Comparons avec une mauvaise prédiction
mauvaise_prediction = {"i": 0.05, "a": 0.7, "e": 0.2, "o": 0.05}
loss_mauvaise = -math.log(mauvaise_prediction[bonne_reponse])
print(
    f"Si le modèle n'avait donné que {mauvaise_prediction[bonne_reponse]:.0%} à '{bonne_reponse}'..."
)
print(f"Loss = {loss_mauvaise:.2f}  (beaucoup plus haut = beaucoup plus faux)")

---
### A toi de jouer ! (Exercice 1)

Dans la cellule ci-dessous, change la valeur de `ma_prediction` pour voir
comment la loss reagit. Essaie `0.9` (tres confiant) et `0.01` (presque sur
que c'est faux).

Plus la prediction est bonne, plus la loss est **basse**.

In [ ]:
# --- EXERCICE 1 : Change la prediction, puis Shift + Entree ---
ma_prediction = 0.6  # <-- Change cette valeur ! Essaie 0.9 ou 0.01

ma_loss = -math.log(ma_prediction)
print(f"Si le modele donne {ma_prediction:.0%} de chance a la bonne reponse :")
print(f"  Loss = {ma_loss:.2f}")
if ma_prediction > 0.8:
    print("  -> Tres bien ! Le modele est confiant et a raison.")
elif ma_prediction < 0.1:
    print("  -> Enorme loss ! Le modele s'est beaucoup trompe.")
else:
    print("  -> Moyen. Le modele peut encore s'ameliorer.")

---
## Étape 2 : Les poids du modèle

Un modèle, c'est juste une collection de **nombres** (on les appelle des **poids**).
Ces nombres déterminent les prédictions.

Entraîner = trouver les bons nombres.

In [ ]:
import math
import random

# On crée un mini-modèle : juste des scores pour chaque paire de lettres
# Au début, les scores sont aléatoires -> le modèle ne sait rien

alphabet = list("abcdefghijklmnopqrstuvwxyz.")

# Scores aléatoires (les "poids" du modèle)
random.seed(42)
poids = {}
for a in alphabet:
    poids[a] = {}
    for b in alphabet:
        poids[a][b] = random.uniform(-1, 1)


def calculer_probas(poids, lettre):
    """Transforme les scores en probabilités (softmax)."""
    scores = poids[lettre]
    # L'exponentielle rend tous les scores positifs
    exps = {b: math.exp(scores[b]) for b in scores}
    total = sum(exps.values())
    return {b: exps[b] / total for b in scores}


# Au début, les probas sont quasi uniformes (le modèle devine au hasard)
p = calculer_probas(poids, ".")
lettres_debut = sorted(p.items(), key=lambda x: -x[1])[:5]
print("Au début, le modèle pense que les Pokémon commencent par :")
for lettre, prob in lettres_debut:
    print(f"  '{lettre}' : {prob:.1%}")
print("\n  -> C'est n'importe quoi ! Il faut l'entraîner.")

---
## Étape 3 : Entraînement

L'algorithme est simple :
1. Prendre un nom de Pokémon d'entraînement
2. Le modèle fait sa prédiction
3. On calcule la loss (l'erreur)
4. On **ajuste les poids** pour réduire la loss
5. Recommencer

L'étape 4 s'appelle la **descente de gradient**. C'est comme ajuster ton
tir au panier un petit peu à chaque essai.

In [ ]:
pokemons = [
    "arcanin",
    "bulbizarre",
    "carapuce",
    "dracaufeu",
    "ectoplasma",
    "evoli",
    "felinferno",
    "gardevoir",
    "goupix",
    "lokhlass",
    "lucario",
    "metamorph",
    "mewtwo",
    "noctali",
    "pikachu",
    "rondoudou",
    "ronflex",
    "salameche",
    "togepi",
    "voltali",
]

# Vitesse d'apprentissage : de combien on ajuste à chaque fois
# Trop grand = on dépasse, trop petit = on apprend trop lentement
vitesse = 0.1  # <-- Change cette valeur ! Essaie 0.01 ou 0.5
nb_epochs = 50  # <-- Change cette valeur ! Essaie 10 ou 200

print("Entraînement...")
print()

for epoch in range(nb_epochs):
    loss_totale = 0
    nb = 0

    for pokemon in pokemons:
        mot = "." + pokemon + "."
        for i in range(len(mot) - 1):
            lettre = mot[i]
            cible = mot[i + 1]

            # 1. Prédiction
            probas = calculer_probas(poids, lettre)

            # 2. Loss
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1

            # 3. Ajuster les poids (gradient simplifié)
            for b in alphabet:
                if b == cible:
                    # La bonne réponse : augmenter son score
                    poids[lettre][b] += vitesse * (1 - probas[b])
                else:
                    # Les mauvaises réponses : baisser leur score
                    poids[lettre][b] -= vitesse * probas[b]

    if epoch % 10 == 0:
        print(f"  Epoch {epoch:2d} | Loss moyenne : {loss_totale / nb:.3f}")

print(f"  Epoch {epoch:2d} | Loss moyenne : {loss_totale / nb:.3f}")
print()
print("La loss baisse = le modèle s'améliore !")

---
### A toi de jouer ! (Exercice 2)

Dans la cellule d'entrainement ci-dessus, change `vitesse` et `nb_epochs`,
puis re-execute-la. Essaie :
- `vitesse = 0.5` : que se passe-t-il ? (instabilite !)
- `vitesse = 0.01` : est-ce que la loss descend aussi vite ?
- `nb_epochs = 200` : est-ce que ca continue a s'ameliorer ?

La cellule ci-dessous fait un mini-test rapide pour comparer :

In [ ]:
# --- EXERCICE 2 : Compare differentes vitesses, puis Shift + Entree ---
import copy

vitesse_test = 0.5  # <-- Change cette valeur ! Essaie 0.01, 0.1, 0.5, 2.0

# On fait un mini-entrainement de 5 epochs avec cette vitesse
poids_test = copy.deepcopy(poids)
for epoch in range(5):
    loss_t = 0
    nb_t = 0
    for pokemon in pokemons:
        mot = "." + pokemon + "."
        for i in range(len(mot) - 1):
            probas_t = calculer_probas(poids_test, mot[i])
            loss_t += -math.log(probas_t[mot[i + 1]] + 1e-10)
            nb_t += 1
            for b in alphabet:
                if b == mot[i + 1]:
                    poids_test[mot[i]][b] += vitesse_test * (1 - probas_t[b])
                else:
                    poids_test[mot[i]][b] -= vitesse_test * probas_t[b]
    print(f"  Epoch {epoch} | vitesse={vitesse_test} | Loss : {loss_t / nb_t:.3f}")

In [ ]:
# Voyons maintenant ce que le modèle a appris :
p = calculer_probas(poids, ".")
lettres_debut = sorted(p.items(), key=lambda x: -x[1])[:5]
print("Après entraînement, les Pokémon commencent par :")
for lettre, prob in lettres_debut:
    print(f"  '{lettre}' : {prob:.1%}")
print()
print("C'est plus logique ! (c, m, g, e, r sont des débuts courants)")

In [ ]:
# Générons des noms de Pokémon avec le modèle entraîné
def generer(poids, n=10):
    resultats = []
    for _ in range(n):
        pokemon = ""
        lettre = "."
        for _ in range(20):  # max 20 lettres
            p = calculer_probas(poids, lettre)
            choix = list(p.keys())
            probs = list(p.values())
            lettre = random.choices(choix, weights=probs, k=1)[0]
            if lettre == ".":
                break
            pokemon += lettre
        if pokemon:
            resultats.append(pokemon.capitalize())
    return resultats


print("Noms de Pokémon inventés après entraînement :")
for p in generer(poids, 10):
    print(f"  {p}")

---
### A toi de jouer ! (Exercice 3)

Dans la cellule ci-dessous, change le nombre pour generer **50** noms.
Est-ce que certains ressemblent a de vrais Pokemon ?

In [ ]:
# --- EXERCICE 3 : Change le nombre, puis Shift + Entree ---
nombre = 10  # <-- Mets 50 ici !

print(f"Generation de {nombre} Pokemon :")
print()
for i, nom in enumerate(generer(poids, nombre)):
    print(f"  {i + 1}. {nom}")

---
## Ce qu'on a appris

- La **loss** mesure à quel point le modèle se trompe
- Les **poids** sont les nombres que le modèle ajuste pour apprendre
- L'**entraînement** = ajuster les poids pour réduire la loss, encore et encore
- Même un modèle simple s'améliore avec l'entraînement !

### Limite

Notre modèle ne regarde encore que **1 lettre en arrière**.
Dans la prochaine leçon, on va lui donner une **mémoire** pour qu'il
se souvienne de plusieurs lettres à la fois.

---
*Prochaine leçon : [03 - La mémoire du modèle](03_la_memoire_du_modele.ipynb)*

---

### Sources (ISO 42001)

- **Cross-entropy loss et descente de gradient** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) — Andrej Karpathy, lignes implémentant le backward pass
- **Analogie du gradient comme correction** : [Vidéo "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) — Andrej Karpathy (2023)
- **Visualisation de la descente de gradient** : [3Blue1Brown - Gradient descent](https://www.youtube.com/watch?v=IHZwWFHWa-w) — Grant Sanderson
- **Dataset Pokémon** : (c) Nintendo / Creatures Inc. / GAME FREAK inc., usage éducatif. Source : [PokéAPI](https://pokeapi.co/)